In [1]:
import sys

sys.path.append("../")
from quicfire_tools import outputs
from quicfire_tools.parameters import SimulationParameters

In [2]:
import zarr
import numpy as np
import xarray as xr
import dask.array as da
from rechunker import rechunk
from pathlib import Path
from IPython.display import display, HTML

In [3]:
TMP_PATH = Path("../tests/tmp")
DATA_PATH = Path("../tests/data")
SIMULATION_PATH = DATA_PATH / "crazy-canyon-simulation"
OUTPUT_PATH = SIMULATION_PATH / "Output"
DRAWFIRE_PATH = OUTPUT_PATH / "drawfire"

# Create simulation parameters object
SIM_PARAMS = SimulationParameters(
    nx=100,
    ny=100,
    nz=56,
    dx=1,
    dy=1,
    dz=1,
    wind_speed=6.5,
    wind_direction=270,
    sim_time=600,
    auto_kill=0,
    num_cpus=4,
    fuel_flag=4,
    ignition_flag=1,
    output_time=100,
    topo_flag=0,
)

In [4]:
sim_outputs = outputs.SimulationOutputs(OUTPUT_PATH, SIM_PARAMS)

In [5]:
zarr_simulation_outputs = sim_outputs.to_zarr(TMP_PATH / "test.zarr")

In [6]:
print(zarr_simulation_outputs.tree())

/
 ├── fire-energy_to_atmos (2, 100, 100, 57) float64
 ├── fire-reaction_rate (2, 100, 100, 57) float64
 ├── fuels-dens (2, 100, 100, 57) float64
 ├── fuels-moist (2, 100, 100, 57) float64
 ├── groundfuelheight (1, 100, 100, 1) float64
 ├── mburnt_integ (2, 100, 100, 1) float64
 ├── thermaldose (2, 100, 100, 57) float64
 ├── thermalradiation (2, 100, 100, 57) float64
 └── windu (2, 100, 100, 57) float64


# Xarray rechunking

### All arrays in a dataset

In [7]:
ds_3d = xr.open_zarr(
    TMP_PATH / "test.zarr",
    drop_variables=["groundfuelheight", "mburnt_integ"],
    consolidated=False,
)

In [8]:
print(ds_3d)

<xarray.Dataset>
Dimensions:               (time: 2, y: 100, x: 100, z: 57)
Dimensions without coordinates: time, y, x, z
Data variables:
    fire-energy_to_atmos  (time, y, x, z) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    fire-reaction_rate    (time, y, x, z) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    fuels-dens            (time, y, x, z) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    fuels-moist           (time, y, x, z) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    thermaldose           (time, y, x, z) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    thermalradiation      (time, y, x, z) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    windu                 (time, y, x, z) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>


In [9]:
ds_3d["fuels-dens"]

<xarray.DataArray 'fuels-dens' (time: 2, y: 100, x: 100, z: 57)>
dask.array<open_dataset-fuels-dens, shape=(2, 100, 100, 57), dtype=float64, chunksize=(1, 100, 100, 57), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x, z

In [10]:
# Rechunk the full dataset along the time dimension
ds_3d_rechunked = ds_3d.chunk({"time": 2})

In [11]:
print(ds_3d_rechunked)

<xarray.Dataset>
Dimensions:               (time: 2, y: 100, x: 100, z: 57)
Dimensions without coordinates: time, y, x, z
Data variables:
    fire-energy_to_atmos  (time, y, x, z) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    fire-reaction_rate    (time, y, x, z) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    fuels-dens            (time, y, x, z) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    fuels-moist           (time, y, x, z) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    thermaldose           (time, y, x, z) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    thermalradiation      (time, y, x, z) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    windu                 (time, y, x, z) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>


In [12]:
ds_3d_rechunked["fuels-dens"]

<xarray.DataArray 'fuels-dens' (time: 2, y: 100, x: 100, z: 57)>
dask.array<rechunk-merge, shape=(2, 100, 100, 57), dtype=float64, chunksize=(2, 100, 100, 57), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x, z

### Rechunk a single data array in the dataset 

In [13]:
da_fuels_dens = ds_3d["fuels-dens"]
da_fuels_dens

<xarray.DataArray 'fuels-dens' (time: 2, y: 100, x: 100, z: 57)>
dask.array<open_dataset-fuels-dens, shape=(2, 100, 100, 57), dtype=float64, chunksize=(1, 100, 100, 57), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x, z

In [14]:
da_fuels_dens_rechunked = da_fuels_dens.chunk({"x": 10, "y": 10, "z": 10})
da_fuels_dens_rechunked

<xarray.DataArray 'fuels-dens' (time: 2, y: 100, x: 100, z: 57)>
dask.array<rechunk-merge, shape=(2, 100, 100, 57), dtype=float64, chunksize=(1, 10, 10, 10), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x, z

### Just the mburnt_integ array in a dataset

In [15]:
output_name = "mburnt_integ"
dropped_variables = [var for var in zarr_simulation_outputs if var != output_name]
ds_2d = xr.open_zarr(
    TMP_PATH / "test.zarr",
    drop_variables=dropped_variables,
    consolidated=False,
)

In [16]:
print(ds_2d)

<xarray.Dataset>
Dimensions:       (time: 2, y: 100, x: 100, z: 1)
Dimensions without coordinates: time, y, x, z
Data variables:
    mburnt_integ  (time, y, x, z) float64 dask.array<chunksize=(1, 100, 100, 1), meta=np.ndarray>


In [17]:
ds_2d["mburnt_integ"]

<xarray.DataArray 'mburnt_integ' (time: 2, y: 100, x: 100, z: 1)>
dask.array<open_dataset-mburnt_integ, shape=(2, 100, 100, 1), dtype=float64, chunksize=(1, 100, 100, 1), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x, z

In [18]:
# Rechunk the ds along the time dimension
ds_2d_rechunked = ds_2d.chunk({"time": -1})

In [19]:
print(ds_2d_rechunked)

<xarray.Dataset>
Dimensions:       (time: 2, y: 100, x: 100, z: 1)
Dimensions without coordinates: time, y, x, z
Data variables:
    mburnt_integ  (time, y, x, z) float64 dask.array<chunksize=(2, 100, 100, 1), meta=np.ndarray>


In [20]:
ds_2d_rechunked["mburnt_integ"]

<xarray.DataArray 'mburnt_integ' (time: 2, y: 100, x: 100, z: 1)>
dask.array<rechunk-merge, shape=(2, 100, 100, 1), dtype=float64, chunksize=(2, 100, 100, 1), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x, z

# Dask rechunking

In [21]:
dask_array = sim_outputs.to_dask("mburnt_integ")

In [22]:
display(HTML(dask_array._repr_html_()))

In [23]:
dask_array_rechunked = dask_array.rechunk((2, 100, 100, 57))

In [24]:
display(HTML(dask_array_rechunked._repr_html_()))

# Zarr rechunking with rechunker

### Rechunk group

In [25]:
zarr_group = sim_outputs.to_zarr(TMP_PATH / "test.zarr", dimensions=True)
target_chunks = {
    "fire-energy_to_atmos": (2, 100, 100, 57),
    "fire-reaction_rate": (2, 100, 100, 57),
    "fuels-dens": (2, 100, 100, 57),
    "fuels-moist": (2, 100, 100, 57),
    "groundfuelheight": (1, 100, 100, 1),
    "mburnt_integ": (2, 100, 100, 1),
    "thermaldose": (2, 100, 100, 57),
    "thermalradiation": (2, 100, 100, 57),
    "windu": (2, 100, 100, 57),
}
max_mem = "100MB"
array_plan = rechunk(
    zarr_group,
    target_chunks,
    max_mem,
    target_store=TMP_PATH / "test_group_rechunked.zarr",
    temp_store=TMP_PATH / "test_group_tmp.zarr",
)
array_plan

<Rechunked>
* Source      : <zarr.hierarchy.Group '/'>

* Intermediate: <zarr.hierarchy.Group '/'>

* Target      : <zarr.hierarchy.Group '/'>

In [26]:
rechunked = array_plan.execute()

In [27]:
print(rechunked)

<zarr.hierarchy.Group '/'>


In [28]:
print(rechunked.tree())

/
 ├── fire-energy_to_atmos (2, 100, 100, 57) float64
 ├── fire-reaction_rate (2, 100, 100, 57) float64
 ├── fuels-dens (2, 100, 100, 57) float64
 ├── fuels-moist (2, 100, 100, 57) float64
 ├── groundfuelheight (1, 100, 100, 1) float64
 ├── mburnt_integ (2, 100, 100, 1) float64
 ├── thermaldose (2, 100, 100, 57) float64
 ├── thermalradiation (2, 100, 100, 57) float64
 └── windu (2, 100, 100, 57) float64


In [29]:
print(rechunked["fuels-dens"].chunks)

(2, 100, 100, 57)


### Rechunk single Store

In [30]:
zarr_group = sim_outputs.to_zarr(TMP_PATH / "test.zarr", dimensions=True)
source_array = zarr_group["fuels-dens"]
target_chunks = (2, 100, 100, 57)
max_mem = "100MB"
array_plan = rechunk(
    source_array,
    target_chunks,
    max_mem,
    target_store=TMP_PATH / "test_store_rechunked.zarr",
    temp_store=TMP_PATH / "test_store_tmp.zarr",
)
array_plan

<Rechunked>
* Source      : <zarr.core.Array '/fuels-dens' (2, 100, 100, 57) float64>

* Target      : <zarr.core.Array (2, 100, 100, 57) float64>

In [31]:
rechunked = array_plan.execute()

In [32]:
print(rechunked)

<zarr.core.Array (2, 100, 100, 57) float64>


In [33]:
print(rechunked.chunks)

(2, 100, 100, 57)
